In [22]:
import os
import time
import numpy as np
import pandas as pd

import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

import keras
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense
     
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [2]:
for dirname, _, filenames in os.walk('C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\columnLabels.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\demographic.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\ERPdata.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\mergedTrialData.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\time.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\1.csv\1.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\10.csv\10.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\11.csv\11.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\12.csv\12.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\13.csv\13.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\14.csv\14.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia\15.csv\15.csv
C:/Users/USER/Downloads/Yusril EEG New Project/eeg

In [3]:
demographic = pd.read_csv("C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia/demographic.csv")

In [4]:
demographic

,subject,group,gender,age,education
0,1,0,M,44,16.0
1,2,0,M,39,17.0
2,3,0,M,53,18.0
3,4,0,M,52,15.0
4,5,0,M,41,16.0
...,...,...,...,...,...
76,77,1,M,28,13.0
77,78,1,F,32,16.0
78,79,1,M,37,16.0
79,80,1,M,33,13.0


In [5]:
demographic[" group"].mean()

0.6049382716049383

In [6]:
demographic[" group"].value_counts()

1    49
0    32
Name:  group, dtype: int64

In [7]:
for i, t in enumerate(list(demographic[" group"])):
    if t:
        print(f"{i} - Schizophrenia")
    else:
        print(f"{i} - HEALTHY")

0 - HEALTHY
1 - HEALTHY
2 - HEALTHY
3 - HEALTHY
4 - HEALTHY
5 - HEALTHY
6 - HEALTHY
7 - HEALTHY
8 - HEALTHY
9 - HEALTHY
10 - HEALTHY
11 - HEALTHY
12 - HEALTHY
13 - HEALTHY
14 - HEALTHY
15 - HEALTHY
16 - HEALTHY
17 - HEALTHY
18 - HEALTHY
19 - HEALTHY
20 - HEALTHY
21 - HEALTHY
22 - HEALTHY
23 - HEALTHY
24 - Schizophrenia
25 - Schizophrenia
26 - Schizophrenia
27 - Schizophrenia
28 - Schizophrenia
29 - Schizophrenia
30 - Schizophrenia
31 - Schizophrenia
32 - Schizophrenia
33 - Schizophrenia
34 - Schizophrenia
35 - Schizophrenia
36 - Schizophrenia
37 - Schizophrenia
38 - Schizophrenia
39 - Schizophrenia
40 - Schizophrenia
41 - Schizophrenia
42 - Schizophrenia
43 - Schizophrenia
44 - Schizophrenia
45 - Schizophrenia
46 - Schizophrenia
47 - Schizophrenia
48 - Schizophrenia
49 - Schizophrenia
50 - Schizophrenia
51 - Schizophrenia
52 - Schizophrenia
53 - Schizophrenia
54 - Schizophrenia
55 - Schizophrenia
56 - Schizophrenia
57 - Schizophrenia
58 - HEALTHY
59 - HEALTHY
60 - HEALTHY
61 - HEALTHY


In [8]:
diagnosis_dict = dict(zip(demographic.subject, demographic[" group"]))
del demographic

In [9]:
electrodes_list = list(pd.read_csv("C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia/columnLabels.csv").columns[4:])
print("Electrodes List \n",electrodes_list)

Electrodes List 
 ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'VEOa', 'VEOb', 'HEOL', 'HEOR', 'Nose', 'TP10']


In [10]:
def averaged_by_N_rows(a, n):
    """ 
    A function that averages n rows in a matrix
    """
    shape = a.shape
    assert len(shape) == 2
    assert shape[0] % n == 0
    b = a.reshape(shape[0] // n, n, 
                  shape[1])
    mean_vec = b.mean(axis=1)
    return mean_vec

In [11]:
N_AVERAGED = 16
X = np.zeros((81 * 100,  9216 * len(electrodes_list) // N_AVERAGED), dtype="float32")
Y = np.zeros(len(X))

part1_path = "C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia"
part2_path = "C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia/buttontonesz2"

# We pull out only those experiments where there were 9216 measurements (most often named so many times)
x_counter = 0
column_list = pd.read_csv("C:/Users/USER/Downloads/Yusril EEG New Project/eeg-schizophrenia/columnLabels.csv").columns
for person_number in tqdm(range(1, 81 + 1)):


    csv_path = f"{part1_path}/{person_number}.csv/{person_number}.csv"
    if not os.path.exists(csv_path):
        csv_path = f"{part2_path}/{person_number}.csv/{person_number}.csv"
    df = pd.read_csv(csv_path, 
                header=None,
                names=column_list)
    trials_list = set(df.trial)


    for t1, trial_number in enumerate(trials_list):
        number_of_trials = len(df[df.trial == trial_number])
        if number_of_trials == 9216.0:
            current_sample_matrix = df[df.trial == trial_number][electrodes_list].values
            averaged_by_N = averaged_by_N_rows(current_sample_matrix, n=N_AVERAGED)
            averaged_by_N_big_vec = averaged_by_N.reshape(-1)
            X[x_counter] = averaged_by_N_big_vec.astype(np.float32)
            Y[x_counter] = diagnosis_dict[person_number]
            x_counter += 1
print("Total trials with the Appropriate number of measurements - ", x_counter)
X = X[: x_counter]
Y = Y[: x_counter]
        
print("Total trials with the Appropriate number of measurements- ", x_counter)


Total trials with the Appropriate number of measurements -  7092
Total trials with the Appropriate number of measurements-  7092


In [12]:
X.shape

(7092, 40320)

In [13]:
Y.shape

(7092,)

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)

In [15]:
X_norm = (normalize(X.reshape(-1, 70), axis=0, norm='max')).reshape(X.shape)

In [16]:
X_train_norm, X_test_norm, Y_train_norm, Y_test_norm = train_test_split(X_norm, Y, test_size=0.2, shuffle=True, random_state=42)

In [17]:
_norm = X

In [18]:
X_train_2d = X_train_norm.reshape(X_train_norm.shape[0], len(electrodes_list), X_train_norm.shape[1] // len(electrodes_list), 1)
X_test_2d = X_test_norm.reshape(X_test_norm.shape[0], len(electrodes_list), X_test_norm.shape[1] // len(electrodes_list), 1)

In [28]:
print(X_train_2d.shape[1:])

(70, 576, 1)


In [19]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 20),
                 activation='tanh',
                 input_shape=(X_train_2d.shape[1:])))
model.add(MaxPooling2D(pool_size=(5, 15)))

model.add(Conv2D(13, kernel_size=(3, 3),
                 activation='tanh',))
model.add(MaxPooling2D(pool_size=(3, 3)))
          

model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(317, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [40]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32, kernel_size=(5, 20),activation='tanh',
                                                            input_shape=(X_train_2d.shape[1:])),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(0.2),
                                      tf.keras.layers.Conv2D(64, (5, 5), activation=tf.nn.relu),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(0.2),
                                      tf.keras.layers.Conv2D(32, (5, 5), activation=tf.nn.relu),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                      tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                      tf.keras.layers.Dense(32, activation=tf.nn.relu),
                                      tf.keras.layers.Dropout(0.2),
                                      tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [41]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 66, 557, 32)       3232      
                                                                 
 batch_normalization_9 (Batc  (None, 66, 557, 32)      128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 33, 278, 32)      0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 33, 278, 32)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 29, 274, 64)       51264     
                                                                 
 batch_normalization_10 (Bat  (None, 29, 274, 64)     

In [42]:
filepath="C:\\Users\\USER\\Downloads\\Yusril EEG New Project\\model2\\{epoch:03d}-val_acc-{val_acc:.3f}.h5"
checkpoint = ModelCheckpoint(
                             filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             mode='auto',
                             #  restore_best_weights=True,
                             )

earlystop = EarlyStopping(monitor='val_loss',
                          mode='auto',
                          patience=4,
                          restore_best_weights=True)

In [43]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tf.optimizers.Adam(0.000008),
              metrics=['acc'])

In [44]:
history = model.fit(X_train_2d, Y_train_norm,
          batch_size=17,
          epochs=1000,
          verbose=1,
          shuffle=True,
          validation_data=(X_test_2d, Y_test_norm), callbacks=checkpoint)

Epoch 1/1000
333/334 [============================>.] - ETA: 0s - loss: 0.7192 - acc: 0.5510
Epoch 00001: val_acc improved from -inf to 0.59126, saving model to C:\Users\USER\Downloads\Yusril EEG New Project\model2\001-val_acc-0.591.h5
334/334 [==============================] - 10s 29ms/step - loss: 0.7192 - acc: 0.5509 - val_loss: 0.7656 - val_acc: 0.5913
Epoch 2/1000
333/334 [============================>.] - ETA: 0s - loss: 0.7065 - acc: 0.5617
Epoch 00002: val_acc did not improve from 0.59126
334/334 [==============================] - 10s 29ms/step - loss: 0.7064 - acc: 0.5618 - val_loss: 0.6895 - val_acc: 0.5708
Epoch 3/1000
333/334 [============================>.] - ETA: 0s - loss: 0.6995 - acc: 0.5621
Epoch 00003: val_acc did not improve from 0.59126
334/334 [==============================] - 10s 29ms/step - loss: 0.6993 - acc: 0.5621 - val_loss: 0.6875 - val_acc: 0.5906
Epoch 4/1000
333/334 [============================>.] - ETA: 0s - loss: 0.6910 - acc: 0.5730
Epoch 00004: val

333/334 [============================>.] - ETA: 0s - loss: 0.6146 - acc: 0.6545
Epoch 00030: val_acc did not improve from 0.60113
334/334 [==============================] - 10s 29ms/step - loss: 0.6147 - acc: 0.6540 - val_loss: 0.8169 - val_acc: 0.5955
Epoch 31/1000
333/334 [============================>.] - ETA: 0s - loss: 0.6133 - acc: 0.6555
Epoch 00031: val_acc improved from 0.60113 to 0.60254, saving model to C:\Users\USER\Downloads\Yusril EEG New Project\model2\031-val_acc-0.603.h5
334/334 [==============================] - 10s 29ms/step - loss: 0.6134 - acc: 0.6557 - val_loss: 0.8074 - val_acc: 0.6025
Epoch 32/1000
333/334 [============================>.] - ETA: 0s - loss: 0.6088 - acc: 0.6584
Epoch 00032: val_acc did not improve from 0.60254
334/334 [==============================] - 10s 29ms/step - loss: 0.6088 - acc: 0.6580 - val_loss: 0.8120 - val_acc: 0.6004
Epoch 33/1000
333/334 [============================>.] - ETA: 0s - loss: 0.6055 - acc: 0.6631
Epoch 00033: val_acc di

Epoch 58/1000
333/334 [============================>.] - ETA: 0s - loss: 0.5442 - acc: 0.7161
Epoch 00058: val_acc did not improve from 0.63002
334/334 [==============================] - 10s 29ms/step - loss: 0.5444 - acc: 0.7158 - val_loss: 0.8090 - val_acc: 0.6138
Epoch 59/1000
333/334 [============================>.] - ETA: 0s - loss: 0.5437 - acc: 0.7152
Epoch 00059: val_acc did not improve from 0.63002
334/334 [==============================] - 10s 29ms/step - loss: 0.5437 - acc: 0.7151 - val_loss: 0.7327 - val_acc: 0.6230
Epoch 60/1000
333/334 [============================>.] - ETA: 0s - loss: 0.5413 - acc: 0.7170
Epoch 00060: val_acc did not improve from 0.63002
334/334 [==============================] - 10s 29ms/step - loss: 0.5409 - acc: 0.7173 - val_loss: 0.8128 - val_acc: 0.6145
Epoch 61/1000
333/334 [============================>.] - ETA: 0s - loss: 0.5285 - acc: 0.7318
Epoch 00061: val_acc did not improve from 0.63002
334/334 [==============================] - 10s 29ms/ste

Epoch 87/1000
333/334 [============================>.] - ETA: 0s - loss: 0.4347 - acc: 0.8009
Epoch 00087: val_acc improved from 0.64693 to 0.64905, saving model to C:\Users\USER\Downloads\Yusril EEG New Project\model2\087-val_acc-0.649.h5
334/334 [==============================] - 10s 29ms/step - loss: 0.4342 - acc: 0.8013 - val_loss: 0.7233 - val_acc: 0.6490
Epoch 88/1000
333/334 [============================>.] - ETA: 0s - loss: 0.4358 - acc: 0.7961
Epoch 00088: val_acc improved from 0.64905 to 0.65187, saving model to C:\Users\USER\Downloads\Yusril EEG New Project\model2\088-val_acc-0.652.h5
334/334 [==============================] - 10s 29ms/step - loss: 0.4355 - acc: 0.7962 - val_loss: 0.7471 - val_acc: 0.6519
Epoch 89/1000
333/334 [============================>.] - ETA: 0s - loss: 0.4222 - acc: 0.7999
Epoch 00089: val_acc did not improve from 0.65187
334/334 [==============================] - 9s 28ms/step - loss: 0.4220 - acc: 0.7999 - val_loss: 0.7258 - val_acc: 0.6483
Epoch 90

Epoch 115/1000
333/334 [============================>.] - ETA: 0s - loss: 0.3301 - acc: 0.8557
Epoch 00115: val_acc did not improve from 0.66667
334/334 [==============================] - 10s 29ms/step - loss: 0.3305 - acc: 0.8553 - val_loss: 0.7792 - val_acc: 0.6646
Epoch 116/1000
333/334 [============================>.] - ETA: 0s - loss: 0.3207 - acc: 0.8603- ETA: 0s - loss: 0.3230 - acc
Epoch 00116: val_acc did not improve from 0.66667
334/334 [==============================] - 9s 28ms/step - loss: 0.3206 - acc: 0.8602 - val_loss: 0.7816 - val_acc: 0.6617
Epoch 117/1000
333/334 [============================>.] - ETA: 0s - loss: 0.3181 - acc: 0.8601
Epoch 00117: val_acc did not improve from 0.66667
334/334 [==============================] - 9s 28ms/step - loss: 0.3178 - acc: 0.8602 - val_loss: 0.8090 - val_acc: 0.6561
Epoch 118/1000
333/334 [============================>.] - ETA: 0s - loss: 0.3140 - acc: 0.8668
Epoch 00118: val_acc did not improve from 0.66667
334/334 [==============

...

Epoch 850/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0012 - acc: 0.9998
Epoch 00850: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0012 - acc: 0.9998 - val_loss: 2.0742 - val_acc: 0.6836
Epoch 851/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0015 - acc: 0.9995
Epoch 00851: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0015 - acc: 0.9995 - val_loss: 2.0372 - val_acc: 0.6836
Epoch 852/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0039 - acc: 0.9986
Epoch 00852: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0039 - acc: 0.9986 - val_loss: 2.1063 - val_acc: 0.6772
Epoch 853/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0030 - acc: 0.9991
Epoch 00853: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/ste

Epoch 881/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0010 - acc: 0.9998
Epoch 00881: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0010 - acc: 0.9998 - val_loss: 2.0734 - val_acc: 0.6871
Epoch 882/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0016 - acc: 0.9995
Epoch 00882: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0016 - acc: 0.9995 - val_loss: 2.1112 - val_acc: 0.6885
Epoch 883/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0028 - acc: 0.9991
Epoch 00883: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0028 - acc: 0.9991 - val_loss: 2.0718 - val_acc: 0.6878
Epoch 884/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0039 - acc: 0.9986
Epoch 00884: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/ste

Epoch 912/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0013 - acc: 0.9995
Epoch 00912: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0013 - acc: 0.9995 - val_loss: 2.0960 - val_acc: 0.6843
Epoch 913/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0013 - acc: 0.9998
Epoch 00913: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0013 - acc: 0.9998 - val_loss: 2.1607 - val_acc: 0.6808
Epoch 914/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0015 - acc: 0.9993- ETA: 0s - loss: 0.0015 - acc: 0.99
Epoch 00914: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0015 - acc: 0.9993 - val_loss: 2.1836 - val_acc: 0.6857
Epoch 915/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0023 - acc: 0.9984
Epoch 00915: val_acc did not improve from 0.69626
334/334 [=========

333/334 [============================>.] - ETA: 0s - loss: 0.0021 - acc: 0.9993
Epoch 00942: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0021 - acc: 0.9993 - val_loss: 2.1586 - val_acc: 0.6716
Epoch 943/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0025 - acc: 0.9991
Epoch 00943: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0025 - acc: 0.9991 - val_loss: 2.1749 - val_acc: 0.6737
Epoch 944/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0012 - acc: 0.9996
Epoch 00944: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0012 - acc: 0.9996 - val_loss: 2.1282 - val_acc: 0.6801
Epoch 945/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0018 - acc: 0.9993
Epoch 00945: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.001

Epoch 973/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0018 - acc: 0.9993
Epoch 00973: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0018 - acc: 0.9993 - val_loss: 2.1308 - val_acc: 0.6786
Epoch 974/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0013 - acc: 0.9996
Epoch 00974: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0013 - acc: 0.9996 - val_loss: 2.1186 - val_acc: 0.6794
Epoch 975/1000
333/334 [============================>.] - ETA: 0s - loss: 0.0016 - acc: 0.9991
Epoch 00975: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms/step - loss: 0.0016 - acc: 0.9991 - val_loss: 2.2209 - val_acc: 0.6829
Epoch 976/1000
333/334 [============================>.] - ETA: 0s - loss: 6.6915e-04 - acc: 1.0000
Epoch 00976: val_acc did not improve from 0.69626
334/334 [==============================] - 9s 28ms